RoadMap- first export search database in RIS

RIS to xlsx

In [ ]:
import os
import re
import pandas as pd

def blank_row(length):
    """Create and return a blank row with the given number of columns."""
    return [None] * length

def convert_ris_to_excel(ris_path):
    """Convert a single RIS file to XLSX, combining repeated fields like AU, KW."""
    xlsx_path = ris_path.replace(".ris", ".xlsx")

    # Read and flatten RIS text
    with open(ris_path, 'r', encoding='utf-8') as ris_file:
        ris_text = ris_file.read().replace("\n", " ")

    # Extract tags and values using regex
    regex = re.compile(r'([A-Z][A-Z0-9])  - (.*?)(?=([A-Z][A-Z0-9])  - |$)')
    matches = re.findall(regex, ris_text)

    if not matches:
        print(f"No valid RIS data found in {os.path.basename(ris_path)}.")
        return

    # Track order of first occurrence of each tag
    headers = []
    seen = set()
    for match in matches:
        tag = match[0]
        if tag not in seen:
            headers.append(tag)
            seen.add(tag)

    column_map = {header: index for index, header in enumerate(headers)}
    data = []
    row = blank_row(len(headers))

    for match in matches:
        ris_id, ris_data = match[0], match[1]
        idx = column_map[ris_id]

        # Append multiple values with separator
        if row[idx]:
            row[idx] += "; " + ris_data
        else:
            row[idx] = ris_data

        # End of one RIS entry
        if ris_id == "ER":
            data.append(row)
            row = blank_row(len(headers))

    if any(row):
        data.append(row)

    # Write to Excel
    df = pd.DataFrame(data, columns=headers)
    df.to_excel(xlsx_path, index=False, engine='openpyxl')
    print(f"Converted: {os.path.basename(ris_path)} → {os.path.basename(xlsx_path)}")

def main():
    """Batch convert all RIS files in a folder to Excel."""
    folder_path = r"C:\Users\xxxxxxx\Data"

    if not os.path.isdir(folder_path):
        print("The specified directory does not exist.")
        return

    ris_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".ris")]

    if not ris_files:
        print("No .ris files found in the directory.")
        return

    for filename in ris_files:
        full_path = os.path.join(folder_path, filename)
        convert_ris_to_excel(full_path)

    print("All RIS files have been successfully converted to Excel.")

if __name__ == "__main__":
    print("Starting RIS to Excel conversion...")
    try:
        main()
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        print(traceback.format_exc())
    finally:
        input("Press <Enter> to close the program.")


xlsl to csv

In [ ]:
import pandas as pd
import os

def convert_excel_file(file_path):
    """
    This function attempts to read an Excel file, repair it if needed, and convert it to CSV.
    """
    # Check if the file exists
    if not os.path.exists(file_path):
        print("Error: File not found.")
        return

    # Try opening as an Excel file
    try:
        df = pd.read_excel(file_path, engine='openpyxl')  # Open with openpyxl
        new_file_path = file_path.replace(".xlsx", "_converted.xlsx")
        
        # Save a repaired copy
        df.to_excel(new_file_path, index=False)
        print(f"Successfully repaired and saved as: {new_file_path}")

        # Also save as CSV
        csv_file_path = file_path.replace(".xlsx", "_converted.csv")
        df.to_csv(csv_file_path, index=False)
        print(f"Converted to CSV: {csv_file_path}")

    except Exception as e:
        print(f"Error reading the Excel file: {e}")

# Specify the file path (Ensure it is properly formatted for Windows)
file_path = r"C:\Users\xxxxxx\Data\new2.csv"

# Run the function
convert_excel_file(file_path)


Merge xlsl

In [ ]:
import os
import pandas as pd

# 1. Folder path and output
folder_path = r"C:\Users\xxxxxxxx\Data"
output_path = os.path.join(folder_path, "Merged_Correctly_Formatted.xlsx")

# 2. Collect all .xls or .xlsx files
xls_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.xls', '.xlsx'))]

if not xls_files:
    print(" No Excel files (.xls/.xlsx) found.")
    exit()

# 3. Get structure from first file
first_df = pd.read_excel(os.path.join(folder_path, xls_files[0]), dtype=str)
columns = first_df.columns.tolist()
merged = pd.DataFrame(columns=columns)

# 4. Merge loop with safety checks
for file in xls_files:
    full_path = os.path.join(folder_path, file)
    try:
        df = pd.read_excel(full_path, dtype=str)
        df = df[df.columns.intersection(columns)]  # Keep only known columns

        if df.shape[1] == 0:
            print(f" Skipped {file}: No matching columns.")
            continue

        df = df[~df.iloc[:, 0].astype(str).str.contains(columns[0], na=False)]  # Remove header-as-row
        df = df.reindex(columns=columns)  # Reorder columns
        merged = pd.concat([merged, df], ignore_index=True)
        print(f" Merged: {file} ({df.shape[0]} rows)")
    except Exception as e:
        print(f" Failed to merge {file}: {e}")

# 5. Save merged output
merged.to_excel(output_path, index=False, engine="openpyxl")
print(f"\n Final merged file saved to: {output_path}")


sometimes you have to merge to retain or extract the columns you want

In [ ]:
import pandas as pd
import os

# List of CSV file paths (replace with actual file paths when running)
file_paths = [
    # Example placeholders — replace with actual file paths
    r"<path_to>/Action1.csv",
    r"<path_to>/Agency1.csv",
    r"<path_to>/Culture1.csv",
    # Add all remaining files here...
]

# Define expected column structure based on RIS tags
expected_columns = ['AB', 'AU', 'DA', 'DO', 'EP', 'ER', 'IS', 'JO', 'KW',
                    'PY', 'SN', 'SP', 'T1', 'T2', 'TY', 'UR', 'VL']

# List to hold cleaned dataframes
dataframes = []

# Loop through each file and process
for file in file_paths:
    try:
        df = pd.read_csv(file, encoding='utf-8-sig', sep=',', engine='python')

        # Skip if dataframe is empty or has too few columns
        if df.empty or len(df.columns) < 3:
            continue

        # Keep only expected columns and enforce consistent order
        df = df.loc[:, df.columns.intersection(expected_columns)]
        df = df.reindex(columns=expected_columns)
        dataframes.append(df)

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Merge all cleaned dataframes
merged_df = pd.concat(dataframes, ignore_index=True)
merged_df.dropna(how='all', inplace=True)
merged_df.drop_duplicates(inplace=True)

# Save the final cleaned dataset
output_path = r"<path_to_output>/MergedData_Cleaned.csv"
merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"Cleaned and merged data saved to: {output_path}")


filter large dataset again with stricter keywords

In [ ]:
import pandas as pd

# Define input and output paths (update with actual paths when running)
input_path = r"<path_to>/Combined_entries_with_doi_exported.csv"
output_path = r"<path_to>/filtered_updated_citations.xlsx"

# Load the dataset
df = pd.read_csv(input_path)

# Define keywords and semantically related terms
search_terms = [
    "decision-making", "decision making", "governance", 
    "urban planning", "spatial planning", 
    "policy formulation", "policy design", "policy development", 
    "artificial intelligence", "machine learning", "AI", "algorithmic"
]

# Optional: limit search to specific fields (e.g., abstract, title, keywords)
search_fields = ['AB', 'T1', 'KW']  # Adjust these column names to match your dataset

# Filter rows where any field contains any of the search terms (case-insensitive)
filtered_df = df[df[search_fields].astype(str).apply(
    lambda x: x.str.contains('|'.join(search_terms), case=False).any(), axis=1)]

# Save results to Excel
filtered_df.to_excel(output_path, index=False)

print(f"Filtered data saved to: {output_path}")


you csn use A combination of keyword search and NLP - BERT(Semantic)

In [ ]:

from sentence_transformers import SentenceTransformer, util

# === 4. Load model ===
print(" Loading transformer model...")
model = SentenceTransformer('all-MiniLM-L6-v2')

The rest of the anlysis depends on what questions you want the database to answer....you can combine a quantititve with Qualitative synthesis...... See an example of such paper [Artfifical intelligent for urban planning governance](https://doi.org/10.1016/j.landurbplan.2025.105337)